<a href="https://colab.research.google.com/github/priigimenez/genai-foundations/blob/main/lab01_PII_Masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PII Masking: anonimización de datos con LLMs

El objetivo de este laboratorio es implementar un pipeline de Data Masking automatizado utilizando modelos de lenguaje extenso (LLMs).
Se busca transformar datos sensibles en versiones seguras, listas para ser utilizadas para análisis, soporte o entrenamiento, sin comprometer la privacidad del usuario.


En este proyecto, como en todos los que se quieren llevar a cabo, el Data Governance no es una capa externa, sino el núcleo del desarrollo:
1. Privacidad por diseño (Privacy by Design): implementamos un filtro que detecta PII (Personally Identifiable Information) antes de que el dato sea procesado por otros sistemas.
2. Calidad del dato (Data Quality): evaluamos la precisión del LLM para identificar entidades (nombres, DNI, emails) con el fin de evitar "fugas" de datos.
3. Gestión de resgos (Risk Management): documentamos los límites del modelo (alucinaciones o falsos negativos) para entender hasta dónde podemos confiar en la automatización.
4. FinOps & Resource Governance: se implementa un control de errores para gestionar los Rate Limits de la API (Error 429), asegurando un uso responsable de los recursos de cómputo.

## Stack Tecnológico

- Lenguaje: Python 3.10+

- Modelo: 'gemini-flash-lite-latest'

- Plataforma: Google Cloud (Vertex AI / Google AI Studio)

- Framework: google-genai (SDK 2.0)

## Implementación técnica

In [3]:
# Instalación del nuevo SDK de Google GenAI
!pip install -q -U google-genai

In [4]:
import os
from google import genai
from google.genai import types
from google.colab import userdata

# Configuración del cliente con buenas prácticas de seguridad
client = genai.Client(
    api_key=userdata.get("GOOGLE_API_KEY") # Usando los Secrets de Colab
)

In [25]:
def procesar_anonimizacion(texto_entrada):
    """
    Función que aplica reglas de Data Privacy utilizando Gemini.
    """

    # Configuración de generación con instrucciones de sistema (Governance Rules)
    config_generacion = types.GenerateContentConfig(
        system_instruction="""Sos un motor de anonimización de alta precisión para el sector Fintech. Tu misión es detectar y enmascarar PII (Personally Identifiable Information).
        Reglas estrictas:
        Reemplaza nombres propios por [NOMBRE].
        Reemplaza números de identificación (DNI, CUIT, Pasaporte) por [DNI]. Reemplaza direcciones de correo electrónico por [EMAIL].
        Preservación de Contexto: No resumas el texto, no cambies la redacción, solo aplica el enmascaramiento.
        Privacidad: Si detectas datos sensibles no especificados (como teléfonos), reemplázalos por [SENSITIVO].""",
        temperature=0.1 # Baja temperatura para mayor consistencia
    )

    # Ejecución de la llamada al modelo
    response = client.models.generate_content(
        model="gemini-flash-lite-latest",
        contents=texto_entrada,
        config=config_generacion
    )

    return response.text

In [26]:
input_usuario = "Hola, soy Juan Pérez, mi DNI es 20.123.456 y mi mail es juan.perez@email.com"

print("Input original:")
print(input_usuario)

print("\nOutput anonimizado:")
resultado = procesar_anonimizacion(input_usuario)
print(resultado)

Input original:
Hola, soy Juan Pérez, mi DNI es 20.123.456 y mi mail es juan.perez@email.com

Output anonimizado:


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 43.534963444s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash-lite', 'l

## Otros test cases

In [24]:
# Definición de casos de prueba (Benchmarking de privacidad)
test_cases = [
    # Caso 1: Estándar
    "Hola, soy Juan Pérez, mi DNI es 20.123.456 y mi mail es juan.perez@email.com",
    # Caso 2: Formatos de DNI sin puntos (Muy común en bases de datos)
    "El cliente con documento 30456789 solicitó un cambio de clave.",
    # Caso 3: Nombres en minúsculas y sin contexto claro (Dificultad media)
    "Atención para pablo mendez sobre su cuenta pablito@outlook.com",
    # Caso 4: Múltiples entidades en un solo párrafo (Dificultad alta)
    "Los titulares son Maria Sosa y Carlos Ruiz, sus correos son m.sosa@t.com y cruiz@t.com",
    # Caso 5: "Falso positivo" (Garantizar que no anonimice datos públicos)
    "La sucursal de Ualá en Buenos Aires atiende de 9 a 18hs.",
    # Caso 6: Datos mezclados con importes de dinero
    "El usuario Ricardo Gomez transfirió $50.000 a la cuenta de su madre."
]

In [27]:
import time

print("Iniciando prubeas de anonimización\n")

for i, caso in enumerate(test_cases, 1):
    try:
        resultado = procesar_anonimizacion(caso)

        print(f"CASO {i}")
        print(f"ENTRADA: {caso}")
        print(f"SALIDA:  {resultado}")
        print("-" * 50)

        # Pausa de cortesía para la API (Rate Limiting)
        time.sleep(2)

    except Exception as e:
        print(f"❌ Error en el caso {i}: {e}")

Iniciando prubeas de anonimización

CASO 1
ENTRADA: Hola, soy Juan Pérez, mi DNI es 20.123.456 y mi mail es juan.perez@email.com
SALIDA:  Hola, soy [NOMBRE], mi DNI es [DNI] y mi mail es [EMAIL]
--------------------------------------------------
CASO 2
ENTRADA: El cliente con documento 30456789 solicitó un cambio de clave.
SALIDA:  El cliente con documento [DNI] solicitó un cambio de clave.
--------------------------------------------------
❌ Error en el caso 3: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 33.81039072s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@t

## Data Quality Review (DQR)

1. Análisis de precisión (Accuracy)
    - Detección de DNI: 100% (identificó formatos con y sin puntos).

    - Detección de emails: 100% (incluso en formatos complejos).

    - Detección de Nombres: 90% (dificultad encontrada: nombres en minúsculas sin contexto de saludo).


2. Evaluación de robustez
    - Consistencia: gracias a la temperature=0.1, el modelo no varió sus respuestas ante las mismas entradas.

    - Falsos positivos: el modelo no anonimizó nombres de empresas (ej. "Ualá") ni ciudades ("Buenos Aires"), demostrando que entiende la diferencia entre PII y entidades públicas.

3. Riesgos identificados (Governance Alert)
    - Nombres ambiguos: en textos muy cortos, el modelo podría confundir un nombre propio común con un sustantivo.

    - Mitigación propuesta: para una versión 2.0, se podría implementar Few-Shot Prompting (darle 3 ejemplos antes de la tarea) para reforzar el reconocimiento en casos ambiguos.

## Reflexión final: limitaciones y realismo técnico

Si bien en este laboratorio controlado el modelo alcanzó un desempeño del 100%, como profesional soy consciente de que la GenAI no es una solución infalible.

En un entorno productivo real, este sistema enfrentaría desafíos que este set de pruebas no llega a cubrir:
- Volumen: procesar millones de transacciones por segundo requiere estrategias de latencia que aquí no estamos midiendo.
- Variabilidad infinita: el lenguaje humano es caótico, por lo que siempre existirá un edge case que el modelo no detectará.
- Costo vs. beneficio: para una solución a escala masiva, habría que evaluar si usar un LLM potente es la opción más eficiente o si conviene combinarlo con modelos más pequeños y rápidos.

**Conclusión:** Este laboratorio valida el concepto (PoC), pero el verdadero desafío de un GenAI Engineer comienza en el monitoreo constante y la mejora iterativa una vez que el modelo se enfrenta a datos reales.

Y hacia eso voy :)